In [5]:
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
client = MongoClient(port=27017)
db=client.insurance
cursor=db.quotes.aggregate([{"$group":{"_id":{"email":"$formData.email","category":"$category","product":"$product"}}},{"$project":{"email":"$_id.email","category":"$_id.category","product":"$_id.product","_id":0}}])

In [6]:
data=pd.DataFrame(cursor)
data["categoryProduct"]=data["category"]+","+data["product"]
categoryProduct=set(data["categoryProduct"])

In [7]:
outputData=list()
for i in categoryProduct:
    iUsers=set(data.email[data["categoryProduct"]==i])    
    for j in categoryProduct:
        curOutput=list()
        curOutput.append(i)
        curOutput.append(j)
        jUsers=set(data.email[data["categoryProduct"]==j])
        curOutput.append(len(iUsers&jUsers))
        outputData.append(curOutput)
outputData=pd.DataFrame(outputData,columns=["i","j","count"])
print(outputData)

                                                i  \
0   Term Life Insurance,Individual Life Insurance   
1   Term Life Insurance,Individual Life Insurance   
2   Term Life Insurance,Individual Life Insurance   
3   Term Life Insurance,Individual Life Insurance   
4                   Motor Insurance,Car Insurance   
5                   Motor Insurance,Car Insurance   
6                   Motor Insurance,Car Insurance   
7                   Motor Insurance,Car Insurance   
8        Health Insurance,Individual Health Guard   
9        Health Insurance,Individual Health Guard   
10       Health Insurance,Individual Health Guard   
11       Health Insurance,Individual Health Guard   
12     Travel Insurance,Overseas Travel Insurance   
13     Travel Insurance,Overseas Travel Insurance   
14     Travel Insurance,Overseas Travel Insurance   
15     Travel Insurance,Overseas Travel Insurance   

                                                j  count  
0   Term Life Insurance,Individual Life

In [26]:
def suggest(category,product):
    client = MongoClient(port=27017)
    db=client.insurance
    cursor=db.quotes.aggregate([{"$group":{"_id":{"email":"$formData.email","category":"$category","product":"$product"}}},{"$project":{"email":"$_id.email","category":"$_id.category","product":"$_id.product","_id":0}}])
    data=pd.DataFrame(cursor)
    data["categoryProduct"]=data["category"]+","+data["product"]
    categoryProduct=set(data["categoryProduct"])
    outputData=list()
    for i in categoryProduct:
        iUsers=set(data.email[data["categoryProduct"]==i])    
        for j in categoryProduct:
            curOutput=list()
            curOutput.append(i)
            curOutput.append(j)
            jUsers=set(data.email[data["categoryProduct"]==j])
            curOutput.append(len(iUsers&jUsers))
            outputData.append(curOutput)
    outputData=pd.DataFrame(outputData,columns=["i","j","count"])
    categoryProduct=category+","+product
    processData=outputData[outputData["i"]==categoryProduct]
    total=sum(processData["count"])
    if(total>0):
        processData["count"]=(processData["count"]*100)/total
        print(processData)
        categories=set(processData.j[processData["count"]>30])
        categoryList=list()
        productList=list()
        for cat in categories:
            split=cat.split(",")
            categoryList.append(split[0])
            productList.append(split[1])
        return list(db.formConfig.find({"category":{"$in":categoryList},"product":{"$in":productList}},{"category":1,"product":1,"image":1,"info":1,"_id":0}))
    return list()

In [28]:
suggest("Travel Insurance","Overseas Travel Insurance")

                                             i  \
12  Travel Insurance,Overseas Travel Insurance   
13  Travel Insurance,Overseas Travel Insurance   
14  Travel Insurance,Overseas Travel Insurance   
15  Travel Insurance,Overseas Travel Insurance   

                                                j   count  
12  Term Life Insurance,Individual Life Insurance  31.250  
13                  Motor Insurance,Car Insurance  31.250  
14       Health Insurance,Individual Health Guard   3.125  
15     Travel Insurance,Overseas Travel Insurance  34.375  


<ipython-input-26-706608aef3f7>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processData["count"]=(processData["count"]*100)/total


[{'image': 'https://images.moneycontrol.com/static-mcnews/2018/03/car-insurance-770x433.jpg',
  'product': 'Car Insurance',
  'category': 'Motor Insurance',
  'info': ''},
 {'image': 'https://www.shriramgi.com/images/travelproductbg.jpg',
  'product': 'Overseas Travel Insurance',
  'category': 'Travel Insurance',
  'info': ''},
 {'image': 'https://brandsite-static.hdfclife.com/media/image/45.jpg',
  'product': 'Individual Life Insurance',
  'category': 'Term Life Insurance',
  'info': ''}]

In [22]:
data[data["email"] in ["rashwinnonda@gmail.com"]]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().